In [10]:
import numpy as np
import pandas as pd
import pickle

import seaborn as sb


from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score



import keras
from keras.models import Sequential
from keras.layers import *
from keras.preprocessing import image
from keras.wrappers.scikit_learn import KerasClassifier

In [4]:
file = open("data/df.pkl",'rb')
df = pickle.load(file)

df

,GSF,Btype,Energy,WeekDay,TAVG,Sem
0,4314.0,Academic,1.402795493602753,1,43.0,0
1,4314.0,Academic,2.6063907146453857,1,43.0,0
2,4314.0,Academic,0.8264167606830597,1,43.0,0
3,4314.0,Academic,0.831238254904747,1,43.0,0
4,4314.0,Academic,4.9978625774383545,1,43.0,0
...,...,...,...,...,...,...
602573,77484.0,Housing,51.58279991149902,1,54.0,0
602574,77484.0,Housing,52.42500019073486,1,54.0,0
602575,77484.0,Housing,50.46094989776611,1,54.0,0
602576,77484.0,Housing,51.54427433013916,1,54.0,0


In [8]:
df.Btype = pd.factorize(df.Btype)[0]

In [9]:

featureData = df.drop(['Energy'],axis=1).values
targetData = df['Energy']

#scaling

scaler = MinMaxScaler(feature_range = (0,1))
scaledfeatureData = pd.DataFrame(scaler.fit_transform(featureData))

In [ ]:
LinReg = LinearRegression(normalize = True)
LinReg.fit(featureData,targetData)

print(LinReg.score(featureData,targetData))

In [ ]:

#One hot encoding Categorical

Btypedf = pd.get_dummies(df['Btype'],prefix='Btype')
dfencoded = pd.concat([df,Btypedf],axis =1)